<a href="https://colab.research.google.com/github/CACC4/My-Jupiter-Notebook/blob/main/clase_word2vec_b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import pandas as pd
import numpy as np
import math
import nltk
import string

remove_punct_map = dict.fromkeys(map(ord, string.punctuation))
pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)
from itertools import compress
from nltk import word_tokenize
import re

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
from nltk.corpus import stopwords
esp_stop = stopwords.words('spanish')
esp_stop = esp_stop + ['ser','tener','deber']
esp_stop.remove('estado')
esp_stop.remove('estados')

In [4]:
esp_stop

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosotros',
 'vosotras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro'

In [5]:
import spacy

In [6]:
from spacy.lang.es.examples import sentences 

In [7]:
!python -m spacy download es_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.9 MB 4.6 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [8]:
nlp = spacy.load('es_core_news_sm')

In [9]:


def lemmatizer(text):  
  doc = nlp(text)
  #return ' '.join([word.lemma_ for word in doc if word.lemma_ not in esp_stop])
  return ' '.join([word.text for word in doc if word.text not in esp_stop])

In [10]:
esp_stop

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosotros',
 'vosotras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro'

https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92


https://github.com/dccuchile/spanish-word-embeddings


### Carga y pre-procesamiento

In [11]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [12]:
cabildos = pd.read_excel("/content/drive/MyDrive/Drive Magister/PNL/Clase 5/resultadocabildoprovincial.xlsx",
)


In [ ]:
cabildos.head()

In [ ]:
cabildos.sample(30)

In [14]:
cabildos.shape

(30414, 12)

Pre-procedimiento (estándar) para un método BoW

In [29]:
cabildos['fundamento'] = cabildos['fundamento'].str.lower()
cabildos['fundamento'] = cabildos['fundamento'].apply(lambda x: re.sub("[0-9]", '', x))
cabildos['fundamento'] = cabildos['fundamento'].str.translate(str.maketrans(' ', ' ', string.punctuation))
cabildos['fundamento'] = cabildos['fundamento'].str.replace('\n',' ')
cabildos['fundamento'] = cabildos['fundamento'].str.strip()


cabildos = cabildos[[len(x.split( )) >3  for x in cabildos.fundamento]]
cabildos = cabildos.reset_index(drop=True)

In [30]:
cabildos.shape

(18530, 12)

### Con gensim

Vamos a partir por entrenar un modelo con el corpus que tenemos. 

In [31]:
import gensim
from gensim import corpora
from gensim import models

Hay que crear primero el corpus, una vez tenemos la data limpia.

In [33]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [34]:
corpus = [word_tokenize(lemmatizer(texto)) for texto in cabildos.fundamento]

In [35]:
print(corpus)

[['descentralizacion', 'autonoma', 'zona', 'extrema', 'pais'], ['toda', 'persona', 'derecho', 'hacer', 'gana'], ['chile', 'recursos', 'naturales', 'capaz', 'conseguir', 'desarrollo', 'actualmente', 'chile', 'parece', 'pais', 'explotado', 'fuentes', 'primarias', 'presentamos', 'industrias'], ['honestos', 'mismos', 'cultura', 'identidad', 'representa', 'mismo'], ['aplica', 'distintos', 'maneras', 'priorizar', 'clase', 'alta', 'justicia', 'igualdad', 'justicia', 'nación', 'transparencia', 'queremos', 'igualdad', 'justicia'], ['empezar', 'avanzar', 'tal', 'modo', 'producir', 'cambio', 'democracia', 'incluye', 'libertad', 'integracion', 'justicia', 'etc', 'poder', 'elegir', 'propia', 'autonomía', 'satisfacer', 'necesidades', 'básicas', 'conciencia', 'tomar', 'decisiones', 'pedir', 'ayuda', 'naciones', 'cambiar'], ['beneficios', 'recursos', 'programas', 'debe', 'necesitan', 'justicia', 'igual', 'existe', 'nivel', 'medio', 'equidad', 'chile', 'clases', 'sociales', 'deberían', 'existir'], ['da

Ahora vamos a entrenar un modelo Word2vec con nuestro corpus. Esto es, vamos a generar un espacio multidimensional que represente las relaciones semánticas de nuestro vocabulario.  

In [36]:
from gensim.models import Word2Vec

In [38]:
model = Word2Vec(corpus, min_count=3,size= 100, window =5, sg = 1)

No es que sea muy útil, pero podemos ver la representación vectorial de alguna palabra del vocabulario...

In [39]:
model.wv['recursos']

array([-9.27458942e-01,  1.51215896e-01, -3.70077565e-02,  8.38973501e-04,
        3.05063903e-01,  4.44463104e-01, -5.94610497e-02, -2.75782764e-01,
        1.70776471e-01, -4.33157086e-02, -1.25567123e-01,  2.35761225e-01,
        9.81314257e-02,  1.02387503e-01,  1.64371863e-01, -2.84602433e-01,
        6.18215725e-02,  1.88662007e-01,  2.10586444e-01, -1.98149920e-01,
       -2.14678273e-01,  1.17007308e-01,  3.21303695e-01, -9.21949521e-02,
       -5.08926690e-01, -3.30490142e-01,  1.95924371e-01, -3.87225866e-01,
        4.12878811e-01, -3.48365963e-01,  1.05646774e-01, -6.85744286e-01,
        1.56929959e-02, -4.06460047e-01, -5.51052749e-01,  3.23005050e-01,
       -6.52691722e-02, -3.11282706e-02,  3.60895455e-01,  3.80286872e-01,
        5.57147682e-01, -3.70779097e-01,  4.14012223e-01, -5.61376214e-01,
        5.45870125e-01, -4.71450776e-01,  1.43564880e-01, -4.83207218e-02,
       -1.75473794e-01,  1.34751759e-03,  4.11425412e-01,  3.76190394e-02,
        1.04558118e-01,  

Así como podemos estimar la similaridad (coseno) entre las representaciones vectoriales de dos palabras

In [40]:
 model.wv.similarity('educación', 'educacion')

0.9256329

In [41]:
 model.wv.similarity('educación', 'salud')

0.81553864

In [42]:
 model.wv.similarity('educación', 'educativo')

0.6663434

In [43]:
 model.wv.similarity('educación', 'trabajo')

0.57592595

Y también podemos buscar las palabras más parecidas a una palabra cualquiera (dentro del vocabulario, claro)

In [45]:
 model.wv.most_similar('recursos')

[('riquezas', 0.8572227358818054),
 ('estratégicos', 0.8393519520759583),
 ('nacionalización', 0.8380873799324036),
 ('recuperar', 0.8314529061317444),
 ('renovables', 0.8292122483253479),
 ('recuperación', 0.8282792568206787),
 ('naturales', 0.8249028325080872),
 ('manos', 0.8213932514190674),
 ('administrados', 0.8211644887924194),
 ('bienes', 0.8176016807556152)]

Si no nos convence nuestro corpus, otra opción es usar un modelo pre-entrado con un gran corpus. Como el que esta acá

https://github.com/dccuchile/spanish-word-embeddings

In [89]:

from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

model2 = KeyedVectors.load_word2vec_format(binary=True)


TypeError: ignored

In [ ]:
from gensim.models import Word2Vec

model2 = gensim.models.KeyedVectors.load_word2vec_format('sbw_vectors.bin', binary=True)

In [ ]:
 model2.most_similar('educación')

[('enseñanza', 0.7887755632400513),
 ('educativa', 0.7624322175979614),
 ('educativo', 0.7323473691940308),
 ('eduación', 0.7299347519874573),
 ('educativos', 0.7249588966369629),
 ('parvularia', 0.7070096135139465),
 ('preescolar', 0.7016569375991821),
 ('educativas', 0.6875496506690979),
 ('enseñaza', 0.6860960721969604),
 ('posprimaria', 0.6720082759857178)]

In [ ]:
 model2.similarity('educación', 'salud')

0.59547806

In [ ]:
 model2.similarity('educación', 'educacion')

0.54617745

In [ ]:
 model2.similarity('casa', 'educativo')

0.27570286

In [ ]:
 model2.most_similar('género')

[('géneros', 0.746495246887207),
 ('subgénero', 0.738937258720398),
 ('genero', 0.6793079376220703),
 ('monotípico', 0.6632034778594971),
 ('Proteuphaedra', 0.6610530018806458),
 ('Gausapia', 0.6553515195846558),
 ('Amerhippus', 0.6463193893432617),
 ('ditrysias', 0.6444892883300781),
 ('Xypetana', 0.6430881023406982),
 ('Tetragnathidae', 0.6430761814117432)]

In [ ]:
 model2.most_similar('sexo')

[('heterosexuales', 0.6806257963180542),
 ('heterosexual', 0.6804140210151672),
 ('homosexual', 0.641345202922821),
 ('intercrural', 0.6283185482025146),
 ('transexualidad', 0.6130891442298889),
 ('sexuales', 0.6102189421653748),
 ('homosexuales', 0.6095353364944458),
 ('tribadismo', 0.6065760254859924),
 ('sexual', 0.604019045829773),
 ('heteroflexibilidad', 0.5982562899589539)]

In [47]:
 model.wv.most_similar('sexo')

[('raza', 0.9961920976638794),
 ('etnia', 0.9901829957962036),
 ('importar', 0.9700226783752441),
 ('religión', 0.9669302701950073),
 ('géneros', 0.962143063545227),
 ('sexual', 0.9606149196624756),
 ('género', 0.9598836302757263),
 ('genero', 0.9575619101524353),
 ('condición', 0.9562788009643555),
 ('discriminación', 0.955249547958374)]

In [ ]:
 model2.most_similar('grande')

[('pequeña', 0.6300310492515564),
 ('pequeño', 0.6242244839668274),
 ('mayor', 0.5784777402877808),
 ('importante', 0.5656843185424805),
 ('enorme', 0.564133882522583),
 ('aglorameración', 0.5611023902893066),
 ('giganticus', 0.5526046752929688),
 ('gran', 0.5491284728050232),
 ('Silmido', 0.5491080284118652),
 ('gigantesco', 0.5438647270202637)]

In [ ]:
 model2.most_similar('gato')

[('perro', 0.7445881366729736),
 ('zorro', 0.7061581611633301),
 ('conejo', 0.7018613815307617),
 ('montés', 0.6875571012496948),
 ('mapache', 0.6867462396621704),
 ('maúlla', 0.6719425916671753),
 ('tigre', 0.6647046804428101),
 ('lybica', 0.6631762385368347),
 ('huiña', 0.6606248617172241),
 ('gatito', 0.6600393056869507)]

In [ ]:
vect1 = model2['rey'] - model2['hombre'] + model2['mujer']

In [ ]:
vect2 = model2['reina']

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarity([vect1],[vect2])

array([[0.77371955]], dtype=float32)

In [ ]:
 model2.similarity('rey', 'hombre')

0.309018

In [ ]:
 model2.similarity('hombre', 'mujer')

0.71744835

In [ ]:
 model2.similarity('rey', 'reina')

0.7253803

In [ ]:
 model2.most_similar('reina')

[('rey', 0.7253803610801697),
 ('princesa', 0.7232583165168762),
 ('consorte', 0.7053413391113281),
 ('Ranavalona', 0.6801236271858215),
 ('monarca', 0.6759358048439026),
 ('Rasoherina', 0.6722511649131775),
 ('reinas', 0.6713809370994568),
 ('Reina', 0.6703052520751953),
 ('Isabel', 0.6599754691123962),
 ('Kottanner', 0.6597455143928528)]

### Clasificación de textos con Word2vec

https://medium.com/@dilip.voleti/classification-using-word2vec-b1d79d375381

In [48]:
reclamos = pd.read_excel("/content/drive/MyDrive/Drive Magister/PNL/Clase 5/reclamos.xlsx",
)


In [49]:
reclamos.head()

,CASO_ID,MERCADO_ANALISTA,DESCRIPCION_CIUDADANO,PETICION_CIUDADANO
0,638428,APIA -Reclamo Seguros,RECIBI UN CORREO ELECTRONICO DONDE SE ME INDIC...,DADO QUE NINGUNA DE LAS CAUSALES ANTERIORES SE...
1,580194,APIA -Reclamo Seguros,Mi nombre es XX y el día 27 de Enero de este a...,Necesito que Metlife me deposite el dinero sol...
2,447321,APIA -Reclamo Seguros,Estimados Señores SVS\n\nCon fecha 13 de Octub...,Por esta razón quiero colocar un reclamo forma...
3,437201,APIA -Reclamo Seguros,En primera instancia solicité renvar mi tarjet...,Solicito devolución de lo que me cobraron por ...
4,385926,Reclamo Valores,reclamo en contra Scotiabank por cobros de man...,condonacion de los cobros que hace el Banco.


In [50]:
reclamos['DESCRIPCION_CIUDADANO'] = reclamos['DESCRIPCION_CIUDADANO'].str.lower()
reclamos['DESCRIPCION_CIUDADANO'] = reclamos['DESCRIPCION_CIUDADANO'].apply(lambda x: re.sub("[0-9]", '', x))
reclamos['DESCRIPCION_CIUDADANO'] = reclamos['DESCRIPCION_CIUDADANO'].str.translate(str.maketrans(' ', ' ', string.punctuation))
reclamos['DESCRIPCION_CIUDADANO'] = reclamos['DESCRIPCION_CIUDADANO'].str.replace('\n',' ')
reclamos['DESCRIPCION_CIUDADANO'] = reclamos['DESCRIPCION_CIUDADANO'].str.strip()


reclamos = reclamos[[len(x.split( )) >2  for x in reclamos.DESCRIPCION_CIUDADANO]]
reclamos = reclamos.reset_index(drop=True)

In [51]:
from sklearn.model_selection import train_test_split

In [52]:
text_train, text_test, y_train, y_test = train_test_split(reclamos['DESCRIPCION_CIUDADANO'], reclamos['MERCADO_ANALISTA'], random_state=42, test_size=0.25)

In [53]:
corpus_train = [word_tokenize(lemmatizer(texto)) for texto in text_train]
corpus_test = [word_tokenize(lemmatizer(texto)) for texto in text_test]

In [54]:
w2v_model = gensim.models.Word2Vec(corpus_train, size=100, window=5, min_count=2)

In [60]:
words = w2v_model.wv.index2entity

In [61]:
len(words)

4620

In [62]:
words

['seguro',
 'respuesta',
 'fecha',
 'xx',
 'seguros',
 'día',
 'siniestro',
 'compañía',
 'si',
 'banco',
 'póliza',
 'pago',
 'año',
 'aseguradora',
 'reclamo',
 'mes',
 'empresa',
 'parte',
 'vehículo',
 'solo',
 'cuenta',
 'carta',
 'meses',
 'hace',
 'días',
 'hacer',
 'mas',
 'contrato',
 'auto',
 'hoy',
 'información',
 'correo',
 'caso',
 'luego',
 'acciones',
 'liquidador',
 'años',
 'taller',
 'dos',
 'uf',
 'presente',
 'mismo',
 'monto',
 'valor',
 'dinero',
 'crédito',
 'momento',
 'poliza',
 'adjunto',
 'embargo',
 'pagar',
 'indica',
 'informe',
 'ahora',
 'después',
 'daños',
 'nunca',
 'fondos',
 'solicitud',
 'aun',
 'octubre',
 'forma',
 'problema',
 'cobertura',
 'julio',
 'situación',
 'según',
 'cuotas',
 'tiempo',
 'dia',
 'persona',
 'estado',
 'hecho',
 'hice',
 'diciembre',
 'marzo',
 'nuevamente',
 'abril',
 'ejecutiva',
 'ninguna',
 'poder',
 'plazo',
 'corredora',
 'vida',
 'dice',
 'total',
 'pasado',
 'además',
 'condiciones',
 'través',
 'svs',
 'agosto',

Ahora generamos la representación vectorial para cada palabra de cada frase de los sets de entrenamiento y prueba

In [63]:
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])  for ls in corpus_train])
X_test_vect  = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])  for ls in corpus_test])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [64]:
X_train_vect.shape

(749,)

In [65]:
X_train_vect[0].shape

(67, 100)

Tenemos entonces un array, donde para cada elemento documento, hay una representación vectorial para cada palabra del documento. Lo que haremos ahora es promediar las representaciones vectoriales para cada documento.

In [66]:
# de https://medium.com/@dilip.voleti/classification-using-word2vec-b1d79d375381
X_train_vect_avg = []
for v in X_train_vect:
   X_train_vect_avg.append(v.mean(axis=0))
        
X_test_vect_avg = []
for v in X_test_vect:
   X_test_vect_avg.append(v.mean(axis=0))


In [67]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf_model = rf.fit(X_train_vect_avg, y_train.values.ravel())

In [68]:
rf_model.score(X_test_vect_avg, y_test)

0.7

#### Usando los vectores del modelo pre-entrenado...

"The original corpus had the following amount of data:

A total of 1420665810 raw words.<br>
A total of 46925295 sentences.<br>
A total of 3817833 unique tokens.<br>

After the skip-gram model was applied, filtering of words with less than 5 occurrences as well as the downsample of the 273 most common words, the following values were obtained:

A total of 771508817 raw words.<br>
A total of 1000653 unique tokens.

The final resource was a corpus of 1000653 word embeddings of dimension 300. "

In [69]:
len(model2.key_to_index)

NameError: ignored

In [ ]:
X_train_vect = np.array([np.array([model2[i] for i in ls if i in model2.key_to_index])  for ls in corpus_train])
X_test_vect  = np.array([np.array([model2[i] for i in ls if i in model2.key_to_index])  for ls in corpus_test])

C:\Users\usuario\AppData\Local\Temp/ipykernel_5436/3489658328.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_vect = np.array([np.array([model2[i] for i in ls if i in model2.key_to_index])  for ls in corpus_train])
C:\Users\usuario\AppData\Local\Temp/ipykernel_5436/3489658328.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test_vect  = np.array([np.array([model2[i] for i in ls if i in model2.key_to_index])  for ls in corpus_test])


In [ ]:
X_train_vect.shape

(749,)

In [ ]:
X_train_vect[0].shape

(72, 300)

Tenemos entonces un array, donde para cada elemento documento, hay una representación vectorial para cada palabra del documento. Lo que haremos ahora es promediar las representaciones vectoriales para cada documento.

In [ ]:
# de https://medium.com/@dilip.voleti/classification-using-word2vec-b1d79d375381
X_train_vect_avg = []
for v in X_train_vect:
   X_train_vect_avg.append(v.mean(axis=0))
        
X_test_vect_avg = []
for v in X_test_vect:
   X_test_vect_avg.append(v.mean(axis=0))


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf_model = rf.fit(X_train_vect_avg, y_train.values.ravel())

In [ ]:
rf_model.score(X_test_vect_avg, y_test)

0.876

Para combinar Word2Vec con Tf-idf:  

https://medium.com/analytics-vidhya/featurization-of-text-data-bow-tf-idf-avgw2v-tfidf-weighted-w2v-7a6c62e8b097